# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
# !pip install langchain
# !pip install -qU langchain-groq
from config import GROQ_API_KEY
# print(GROQ_API_KEY)

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [4]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [5]:
#!pip install pandas

## LLMChain

In [6]:
# from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [7]:
# llm = ChatOpenAI(temperature=0.9, model=llm_model)
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

In [8]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [9]:
chain = LLMChain(llm=llm, prompt=prompt)

C:\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [10]:
product = "Queen Size Sheet Set"
chain.run(product)

C:\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Here are some name suggestions for a company that makes Queen Size Sheet Set:\n\n1. **Regal Linens**: "Regal" conveys a sense of luxury and grandeur, perfect for a company that produces high-quality sheet sets.\n2. **Queenly Bedding**: This name incorporates the idea of a queen-size bed, while also emphasizing the company\'s focus on bedding.\n3. **Soft Haven**: "Soft" evokes a sense of comfort and coziness, while "Haven" suggests a safe and welcoming place to rest.\n4. **Dreamweave**: This name has a dreamy, whimsical quality to it, implying that the company\'s sheets will help customers sleep better and have sweet dreams.\n5. **Luxury Rest**: This name emphasizes the comfort and luxury of the sheet sets, positioning the company as a provider of high-end bedding.\n6. **Queen\'s Quilted**: This name incorporates the idea of a queen-size bed, while also highlighting the company\'s focus on quilting and craftsmanship.\n7. **Slumber & Co.**: This name has a friendly, approachable tone, w

## SimpleSequentialChain

In [11]:
from langchain.chains import SimpleSequentialChain

In [12]:
# llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [13]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [14]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [15]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
What a great question!

Here are some name suggestions for a company that makes Queen Size Sheet Set:

1. **Regal Bedding Co.**: "Regal" conveys a sense of luxury and high quality, which is perfect for a company that produces high-quality sheet sets.
2. **Queenly Linens**: This name incorporates the idea of the Queen size bed sheets, while also emphasizing the linens aspect.
3. **Dreamweave**: This name suggests that the company's products are woven with care and attention to detail, creating a dreamy sleeping experience.
4. **Sleep Sanctuary**: This name evokes a sense of a cozy, peaceful sleeping space, which is exactly what customers are looking for in a sheet set.
5. **Her Majesty's Linens**: A playful nod to the Queen size bed sheets, while also referencing the idea of high-quality, regal linens.
6. **SlumberSoft**: This name emphasizes the comfort and softness of the sheet sets, which is a key selling point for customers.
7. **Luxe 

'Here is a 20-word description for Regal Bedding Co.:\n\n"Regal Bedding Co. crafts luxurious Queen Size Sheet Sets with exceptional quality, comfort, and style, elevating your sleep experience to royal heights."'

## SequentialChain

In [16]:
from langchain.chains import SequentialChain

In [17]:
# llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [18]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [19]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [20]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [21]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [22]:
review = """Hertz, fácil de encontrar con excelente servicio. Alquilamos un Auto DS (Citroen), este es un vehículo excepcional y es un placer conducirlo. El mejor sistema de navegación que he usado y perfecto. El precio era un poco alto pero valió la pena.
Cliente feliz."""
overall_chain(review)



> Entering new SequentialChain chain...


C:\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



> Finished chain.


{'Review': 'Hertz, fácil de encontrar con excelente servicio. Alquilamos un Auto DS (Citroen), este es un vehículo excepcional y es un placer conducirlo. El mejor sistema de navegación que he usado y perfecto. El precio era un poco alto pero valió la pena.\nCliente feliz.',
 'English_Review': 'Here is the translation of the review to English:\n\n"Hertz, easy to find with excellent service. We rented a DS Auto (Citroen), and this is an exceptional vehicle, a pleasure to drive. The best navigation system I\'ve used and perfect. The price was a bit high, but it was worth it.\nHappy customer."',
 'summary': 'Here is a summary of the review in 1 sentence:\n\nThe reviewer had a positive experience with Hertz, finding the car easy to locate and receiving excellent service, and particularly enjoyed driving the Citroen DS Auto with its excellent navigation system.',
 'followup_message': 'Here is a potential follow-up response in Spanish:\n\n"Estoy de acuerdo con la opinión del revisor. También 

## Router Chain

In [23]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [24]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [25]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [26]:
# llm = ChatOpenAI(temperature=0, model=llm_model)

In [27]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [28]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [29]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [30]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [31]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [32]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation in the context of thermodynamics?'}
> Finished chain.


'Black body radiation is a fundamental concept in thermodynamics that describes the electromagnetic radiation emitted by an idealized object, known as a black body, as it absorbs and re-emits thermal energy.\n\nIn simple terms, a black body is an object that absorbs all incident electromagnetic radiation, without reflecting or transmitting any of it. This means that it has a perfect absorptivity, or a "black" color.\n\nWhen a black body is heated, it begins to emit radiation due to the thermal motion of its particles. This radiation is characterized by a specific distribution of energy, known as the Planck distribution, which depends on the temperature of the black body.\n\nThe Planck distribution is a mathematical function that describes the energy density of the radiation as a function of frequency. It shows that the radiation is not uniform, but instead has a characteristic shape, with more energy emitted at lower frequencies and less energy emitted at higher frequencies.\n\nBlack b

In [33]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


"A simple question!\n\nTo answer this, I'll break it down into its component parts:\n\n* What is 2?\nAnswer: 2 is a number.\n* What is +?\nAnswer: + is the operation of addition, which combines two numbers to get their sum.\n* What is 2 again?\nAnswer: 2 is the same number as before.\n\nNow, let's combine the parts:\n\n* 2 (the first number) + 2 (the second number) = ?\n\nTo add these two numbers, I'll follow the standard procedure for addition:\n\n* Start with the first number: 2\n* Add the second number: 2\n* Combine the two numbers: 2 + 2 = 4\n\nSo, the answer to 2 + 2 is... 4!"

In [35]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
physics: {'input': 'Why does every cell in our body contain DNA? Is it related to the structure of cells?'}
> Finished chain.


'What a great question!\n\nThe reason why every cell in our body contains DNA is not directly related to the structure of cells, but rather it\'s a fundamental property of life.\n\nYou see, DNA (Deoxyribonucleic acid) is the molecule that contains the genetic instructions used in the development and function of all living organisms. It\'s often referred to as the "blueprint" or "instruction manual" for life.\n\nThe reason why DNA is present in every cell is because it\'s necessary for the cell to function properly. DNA contains the genetic information that determines the characteristics of an organism, such as eye color, hair color, height, and other traits. It also contains the instructions for the cell to grow, divide, and respond to its environment.\n\nIn other words, DNA provides the genetic blueprint for the cell to become what it is meant to be. Without DNA, cells wouldn\'t be able to function properly, and life as we know it wouldn\'t be possible.\n\nThe structure of cells, on t

Reminder: Download your notebook to you local computer to save your work.